<a href="https://colab.research.google.com/github/zgordounc/pollock/blob/main/Front_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install docx2txt
!pip install transformers
!pip install torch
!pip install gradio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3977 sha256=f879746770eb7bbef5014061d784bf423965beac42fb6fede5e9270b6453dd91
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import re
import gradio as gr
import os
import docx2txt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")
model = AutoModelForSeq2SeqLM.from_pretrained("nalmy/pollock-590").to(device)

def summarize(text_file):
    file_extension = os.path.splitext(text_file.name)[1]
    if file_extension == ".txt":
        # Load text from a txt file
        with open(text_file.name, "r", encoding='latin1') as f:
          text = f.read()
    elif file_extension == ".docx":
        # Load text from a Word file
        text = docx2txt.process(text_file.name)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

    #input_ids = tokenizer(text, return_tensors="pt").input_ids
    #global_attention_mask = torch.zeros_like(input_ids)
    # set global_attention_mask on first token
    #global_attention_mask[:, 0] = 1

    inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    inputs = inputs.to(device)
    summary_ids = model.generate(inputs)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


    
    return text, summary


iface = gr.Interface(
    fn=summarize,
    inputs=gr.inputs.File(label="Upload a txt file or a Word file for the input text"),
    outputs=[gr.outputs.Textbox(label="Original text"), gr.outputs.Textbox(label="Summary")],
    title="Academic Paper Summarization Demo",
    description="Upload a txt file or a Word file for the input text. Get a summary generated by a small T5 model from Hugging Face.",
)

iface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:347: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `keep_filename` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:

iface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>